In [1]:
val sc = spark.sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
77,application_1613059669625_0003,spark,idle,Link,Link


SparkSession available as 'spark'.
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4cae4aad


In [2]:
import org.apache.spark.sql.types.{FloatType,LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.types.{FloatType, LongType}
import org.apache.spark.sql.types.{ArrayType, StructField, StructType}
import org.apache.spark.sql.functions.explode
import org.apache.spark.sql.expressions.Window


In [3]:
import com.logicalclocks.hsfs._

import com.logicalclocks.hsfs._


In [4]:
val connection = HopsworksConnection.builder().build();
val fs = connection.getFeatureStore();

connection: com.logicalclocks.hsfs.HopsworksConnection = com.logicalclocks.hsfs.HopsworksConnection@7c61b14a
fs: com.logicalclocks.hsfs.FeatureStore = FeatureStore{id=1092, name='amldemo_featurestore', projectId=1144, featureGroupApi=com.logicalclocks.hsfs.metadata.FeatureGroupApi@5fdf9eac}


In [5]:
val df_all = fs.getFeatureGroup("simts_features", 1).read()

df_all: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [f_2: double, f_4: double ... 364 more fields]


In [6]:
df_all.count()

res1: Long = 60000


In [7]:
df_all.where($"target" === 1).count()

res2: Long = 1000


In [8]:
df_all.where($"target" === 0).count()

res3: Long = 59000


In [9]:
print(df_all.columns.toSeq)

WrappedArray(f_2, f_4, f_12, f_21, f_23, f_27, f_30, f_32, f_35, f_37, f_38, f_39, f_52, f_53, f_54, f_57, f_63, f_65, f_69, f_82, f_87, f_99, f_106, f_111, f_118, f_123, f_126, f_127, f_131, f_133, f_140, f_142, f_143, f_146, f_152, f_155, f_157, f_163, f_169, f_170, f_174, f_182, f_184, f_186, f_191, f_196, f_201, f_203, f_214, f_215, f_220, f_221, f_222, f_225, f_229, f_234, f_242, f_243, f_244, f_249, f_256, f_258, f_259, f_274, f_275, f_277, f_281, f_282, f_285, f_291, f_297, f_298, f_306, f_308, f_309, f_310, f_314, f_320, f_322, f_325, f_334, f_335, f_351, f_358, target, f_1, f_7, f_10, f_20, f_22, f_24, f_33, f_34, f_41, f_44, f_46, f_47, f_49, f_51, f_58, f_70, f_74, f_76, f_80, f_81, f_83, f_91, f_92, f_93, f_97, f_110, f_114, f_120, f_121, f_128, f_130, f_132, f_135, f_139, f_145, f_147, f_148, f_149, f_151, f_154, f_159, f_161, f_164, f_168, f_172, f_175, f_178, f_179, f_181, f_185, f_188, f_190, f_193, f_194, f_195, f_197, f_206, f_207, f_212, f_213, f_218, f_224, f_226, f

In [10]:
print(df_all.columns.slice(0,365).map(a => a.toString ).toSeq)

WrappedArray(f_2, f_4, f_12, f_21, f_23, f_27, f_30, f_32, f_35, f_37, f_38, f_39, f_52, f_53, f_54, f_57, f_63, f_65, f_69, f_82, f_87, f_99, f_106, f_111, f_118, f_123, f_126, f_127, f_131, f_133, f_140, f_142, f_143, f_146, f_152, f_155, f_157, f_163, f_169, f_170, f_174, f_182, f_184, f_186, f_191, f_196, f_201, f_203, f_214, f_215, f_220, f_221, f_222, f_225, f_229, f_234, f_242, f_243, f_244, f_249, f_256, f_258, f_259, f_274, f_275, f_277, f_281, f_282, f_285, f_291, f_297, f_298, f_306, f_308, f_309, f_310, f_314, f_320, f_322, f_325, f_334, f_335, f_351, f_358, target, f_1, f_7, f_10, f_20, f_22, f_24, f_33, f_34, f_41, f_44, f_46, f_47, f_49, f_51, f_58, f_70, f_74, f_76, f_80, f_81, f_83, f_91, f_92, f_93, f_97, f_110, f_114, f_120, f_121, f_128, f_130, f_132, f_135, f_139, f_145, f_147, f_148, f_149, f_151, f_154, f_159, f_161, f_164, f_168, f_172, f_175, f_178, f_179, f_181, f_185, f_188, f_190, f_193, f_194, f_195, f_197, f_206, f_207, f_212, f_213, f_218, f_224, f_226, f

In [11]:
import org.apache.spark.ml.feature.{MinMaxScaler,StandardScaler,Normalizer, MaxAbsScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().setInputCols(df_all.columns.slice(0,365).map(a => a.toString).toArray).setOutputCol("features")

val df_all_features = assembler.transform(df_all)

// scaler instance with the min(-1) and max(1) 
val mms = new MinMaxScaler().
    setInputCol("features").
    setOutputCol("mms").
    setMax(1).
    setMin(-1)


val mms_tr = mms.setInputCol("features").setOutputCol("mms")

val trainingFeaturesPipeline = (new Pipeline()
  .setStages(Array(mms_tr)))

val trainingFeaturesDF = trainingFeaturesPipeline.fit(df_all_features).transform(df_all_features)

import org.apache.spark.ml.feature.{MinMaxScaler, StandardScaler, Normalizer, MaxAbsScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.VectorAssembler
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_2abe75f13a82
df_all_features: org.apache.spark.sql.DataFrame = [f_2: double, f_4: double ... 365 more fields]
mms: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_0c85f2163fad
mms_tr: mms.type = minMaxScal_0c85f2163fad
trainingFeaturesPipeline: org.apache.spark.ml.Pipeline = pipeline_3a2a14187f74
trainingFeaturesDF: org.apache.spark.sql.DataFrame = [f_2: double, f_4: double ... 366 more fields]


In [12]:
import org.apache.spark.ml.linalg.DenseVector
val toArr: Any => Array[Double] = _.asInstanceOf[DenseVector].toArray
val toArrUdf = udf(toArr)

val final_df_all = trainingFeaturesDF.select($"target",$"mms").
             withColumn("target",$"target".cast(FloatType)).
             withColumn("mms",toArrUdf($"mms").cast(ArrayType(FloatType)))

import org.apache.spark.ml.linalg.DenseVector
toArr: Any => Array[Double] = <function1>
toArrUdf: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)
final_df_all: org.apache.spark.sql.DataFrame = [target: float, mms: array<float>]


In [13]:
val van = final_df_all.where($"target"===1)
val ben = final_df_all.where($"target"===0) 

van: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]
ben: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]


In [14]:
// Now that the data has been prepared, let's split the dataset into a training and test dataframe
val Array(benTrainDF, benEvalDF) = final_df_all.randomSplit(Array(0.8, 0.02),seed = 12345)

benTrainDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]
benEvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]


In [15]:
benTrainDF.count()

res16: Long = 58539


In [16]:
benEvalDF.count()

res17: Long = 1461


In [17]:
val EvalDF = benEvalDF.union(van)

EvalDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]


In [18]:
EvalDF.where($"target"===1).count()

res18: Long = 1024


In [19]:
EvalDF.where($"target"===0).count()

res19: Long = 1437


In [20]:
val ben_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("ben_td")
                .description("Ben simulated Time Series Training Dataset")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
ben_sim_ts_td_meta.save(benTrainDF)

ben_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@198389d3


In [21]:
val eval_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("eval_td")
                .description("Simulated Time Series Training Dataset for evaluation")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
eval_sim_ts_td_meta.save(EvalDF)

eval_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@77dd83a6


In [22]:
val testDf = EvalDF.where($"target"===1)

val test_sim_ts_td_meta = (fs.createTrainingDataset()
                .name("test_td")
                .description("Simulated Time Series Training Dataset for testing")
                .dataFormat(DataFormat.TFRECORD)     
                .statisticsEnabled(false)          
                .version(1)
                .build())
test_sim_ts_td_meta.save(testDf)

testDf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [target: float, mms: array<float>]
test_sim_ts_td_meta: com.logicalclocks.hsfs.TrainingDataset = com.logicalclocks.hsfs.TrainingDataset@1f276e26
